In [1]:
import open3d as o3d
import numpy as np
import os
import json
import random

def show(name_dict, folder, show_opts):    
    pcd_pts = []
    pcd_color = []
    
    obj_str = f"_{name_dict['object_id']}_{name_dict['object_name']}_{name_dict['ann_id']}_{name_dict['score']}_{name_dict['iou']}"
    
    for opt, v in show_opts.items():
        if v:
            if opt == 'gt':
                path = os.path.join(folder, f"gt_{name_dict['object_id']}_{name_dict['object_name']}.ply")
            elif opt == 'scene':
                path = os.path.join(folder, "pc.ply")
            else:
                path = os.path.join(folder, opt + obj_str + ".ply")
                
            pcd = o3d.io.read_point_cloud(path)
            pts = np.asarray(pcd.points)
            color = np.asarray(pcd.colors)
            pcd_pts.append(pts)
            pcd_color.append(color)
            
            if "tpm" in path and "box" not in path:
                c = np.array([251, 158, 5]) / 256
                color[:, 0] = c[0]
                color[:, 1] = c[1]
                color[:, 2] = c[2]
                
            if "sample" in path:
                c = np.array([174, 66, 200]) / 256
                color[:, 0] = c[0]
                color[:, 1] = c[1]
                color[:, 2] = c[2]

    pcd_pts = np.concatenate(pcd_pts, axis=0)
    pcd_color = np.concatenate(pcd_color, axis=0)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_pts)
    pcd.colors = o3d.utility.Vector3dVector(pcd_color)

    o3d.visualization.draw_geometries([pcd])
    
def get_proposal_list(folder, scene, label_path):
    labels = json.load(open(label_path, 'r'))
    proposal_list = []
    for file in os.listdir(os.path.join(folder, scene)):
        if "pred" in file:
            split_str = file.replace(".ply", "").split("_")
            data_dict = {}
            data_dict['object_id'] = split_str[1]
            data_dict['object_name'] = '_'.join(split_str[2:-3])
            data_dict['ann_id'] = split_str[-3]
            data_dict['score'] = split_str[-2]
            data_dict['iou'] = split_str[-1]
            for label in labels:
                if label['scene_id'] == scene and label['object_id'] == data_dict['object_id'] and label['ann_id'] == data_dict['ann_id']:
                    data_dict['description'] = label['description']
                    break
            proposal_list.append(data_dict)
            
    return proposal_list

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
folder = "author"
scene = "scene0011_00"
label_path = "ScanRefer_filtered_val.json"

proposals = get_proposal_list(folder, scene, label_path)


In [4]:
scene_list = os.listdir("author")[:5]
total = 0
zero = 0
for f in scene_list:
    proposals = get_proposal_list(folder, f, label_path)
    for p in proposals:
        if float(p['iou']) < 0.001:
            zero += 1
        total += 1
print("total scene num: ", len(scene_list))
print(f"zero {zero} out of total {total} -- {zero/total}")

total scene num:  1
zero 53 out of total 76 -- 0.6973684210526315


In [5]:
scene_list = sorted(os.listdir("author3"))[:5]
print(scene_list)
folder = "author3"


def get_bbox(name_dict, folder):
    obj_str = f"_{name_dict['object_id']}_{name_dict['object_name']}_{name_dict['ann_id']}_{name_dict['score']}_{name_dict['iou']}"
    path = os.path.join(folder, 'pred' + obj_str + ".ply")
                
    pcd = o3d.io.read_point_cloud(path)
    pts = np.asarray(pcd.points)
    
    bbox_max = np.max(pts, axis=0)
    bbox_min = np.min(pts, axis=0)
    
    return bbox_max, bbox_min

def get_tpm_points(name_dict, folder):
    obj_str = f"_{name_dict['object_id']}_{name_dict['object_name']}_{name_dict['ann_id']}_{name_dict['score']}_{name_dict['iou']}"
    path = os.path.join(folder, 'tpm1' + obj_str + ".ply")
                
    pcd = o3d.io.read_point_cloud(path)
    pts = np.asarray(pcd.points)
    
    return pts

out = 0
total = 0
for s in scene_list:
    proposals = get_proposal_list(folder, s, label_path)
    for p in proposals:
        bbox_max, bbox_min = get_bbox(p, os.path.join(folder, s))
        pts = get_tpm_points(p, os.path.join(folder, s))
        for pt in pts:
            if pt[0] < bbox_min[0] or pt[0] > bbox_max[0] \
               or pt[1] < bbox_min[1] or pt[1] > bbox_max[1] \
               or pt[2] < bbox_min[2] or pt[2] > bbox_max[2]:
                   out += 1
            total += 1
        
out / total


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'author3'

In [4]:
folder = "author"
scene = "scene0616_00"
label_path = "ScanRefer_filtered_val.json"
proposals = get_proposal_list(folder, scene, label_path)


find_id, find_ann = 5, 2
# find_id, find_ann = None, None

proposal = None
if find_id is not None:
    for p in proposals:
        if p['object_id'] == str(find_id) and p['ann_id'] == str(find_ann):
            proposal = p
else:
    proposal = proposals[random.randint(0, len(proposals)-1)]


print("scene: ", scene)

show_opts = {}
show_opts['gt'] = True
show_opts['pred'] = True

show_opts['filter'] = False
show_opts['tpm0'] = False
show_opts['tpm1'] = False
show_opts['tpm2'] = False
show_opts['tpm3'] = True
show_opts['sample'] = False
show_opts['seed'] = False

show_opts['tpmbox0'] = False
show_opts['tpmbox1'] = False
show_opts['tpmbox2'] = False
show_opts['tpmbox3'] = False

show_opts['scene'] = True


print(proposal['description'])
show(proposal, os.path.join(folder, scene), show_opts)
proposal

scene:  scene0616_00
a table sits in the corner by a brightly colored wall. it's got two chairs facing it.


{'object_id': '5',
 'object_name': 'table',
 'ann_id': '2',
 'score': '0.08717',
 'iou': '0.71041',
 'description': "a table sits in the corner by a brightly colored wall. it's got two chairs facing it."}